In [ ]:
library(tidyverse)
library(stringr) 
library(plotly)

In [ ]:
##############################################################################

##########################     INPUT PARAMETERS    ###########################

##############################################################################



# filetype = # "ratio_all_germline_singletons_excluded_tumor_4_cs" # # "ratio_singletons_excluded_tumor_4_cs" # "ratio" 
filetype = "ratio"

# input_folder =   "ci5_cs1e9" # "all_germline_ci5_cs1e9" # "all_germline_filtered_bams_tumor_ci5_cs1e9" # "all_germline_filtered_bams_tumor_subtract_other_tumor_ci5_cs1e9", all_germline_filtered_bams_tumor_ci5_cs1e9_filtered_cfDNA
# all_germline_filtered_bams_tumor_ci5_cs1e9_filtered_cfDNA # de_novo_germline # all_germline_filtered_bams_tumor_ci5_cs1e9_filtered_cfDNA_subtracted_kmers_not_seen
# all_germline_filtered_bams_tumor_ci5_cs1e9_well_mapping_subtracted # de_novo_germline # de_novo_germline_filtered_cfDNA # de_novo_germline_reference
#input_folder = "de_novo_germline"
input_folder = "final_pipeline_NEW"

# output_folder =  "ci5_cs1e9" # "union_germline_ci4_cs1e9" # "union_germline_filtered_tumor_bams_ci5_cs1e9" # "union_germline_filtered_tumor_bams_subtract_other_tumors_ci5_cs1e9"
# union_germline_filtered_tumor_bams_ci5_cs1e9_filtered_cfDNA # de_novo_germline # union_germline_filtered_bams_tumor_ci5_cs1e9_filtered_cfDNA_subtracted_kmers_not_seen
# union_germline_filtered_tumor_ci5_cs1e9_well_mapping_subtracted # de_novo_germline # de_novo_germline_filtered_cfDNA # de_novo_germline_reference
output_folder = "final_pipeline_pII"

low_qual_sample <- read.table("../low_qual_sample.txt")
low_qual_sample <- as.character(low_qual_sample[[1]])

In [ ]:
paramspace = read.csv("../data/metadata/paramspace_cfDNA_phaseII_missing_removed.csv")

paramspace  <- paramspace %>% rename(patient_id = pt_id)
rm_pt <- read.csv("../phase2_exclude3.csv", header=FALSE)
rm_pt <- as.character(unlist(c(rm_pt[1,])))
paramspace <- paramspace %>% filter(!patient_id %in% rm_pt)

head(paramspace)

In [ ]:
SAMPLES = paramspace["patient_id"]
FOLDERS = paramspace["cfDNA_folder"]

In [ ]:
dim(paramspace)

In [ ]:
#paramspace["big_ratio"] <- NA
#paramspace["big_ratio_CI_lower"] <- NA
#paramspace["big_ratio_CI_upper"] <- NA

paramspace["small_ratio"] <- NA
paramspace["small_ratio_CI_lower"] <- NA
paramspace["small_ratio_CI_upper"] <- NA

#paramspace["jaccard"] <- NA
#paramspace["jaccard_CI_lower"] <- NA
#paramspace["jaccard_CI_upper"] <- NA

for (row in 1:nrow(paramspace)){
    pt = paramspace[row, "patient_id"]
    #print(pt)
    fd = paramspace[row, "cfDNA_folder"]
    #big_r = read.csv(paste("../data/", pt, "/" , fd, "/", input_folder, "/", "big_", filetype, ".csv", sep = ""))
    #jaccard = read.csv(paste("../data/", pt, "/" , fd, "/", input_folder, "/", "jaccard_index.csv", sep = ""))
    small_r = read.csv(paste("../data/phase_II_patients/", pt, "/", fd, "/", input_folder, "/", "small_", filetype, ".csv", sep = ""))
    #print(big_r[[1]])
    #print(small_r[[1]])
    #paramspace[row, "big_ratio"] = big_r$ratio
    #paramspace[row, "big_ratio_CI_lower"] = big_r$lower_CI
    #paramspace[row, "big_ratio_CI_upper"] = big_r$upper_CI

    #paramspace[row, "jaccard"] = jaccard$ratio
    #paramspace[row, "jaccard_CI_lower"] = jaccard$lower_CI
   # paramspace[row, "jaccard_CI_upper"] = jaccard$upper_CI
    
    paramspace[row, "small_ratio"] = small_r$ratio
    paramspace[row, "small_ratio_CI_lower"] = small_r$lower_CI
    paramspace[row, "small_ratio_CI_upper"] = small_r$upper_CI
}

In [ ]:
res <- paramspace
head(res)

In [ ]:
#res <- res %>% separate(cfDNA_folder, c("sampleID", "sample_type", "sample_number"))
res <- res %>% 
  mutate(tmp_chunks = stringr::str_split(cfDNA_folder, stringr::fixed("_"),  n = 3)) %>%
  mutate(sampleID = map_chr(tmp_chunks, 1),
         sub_value = map_chr(tmp_chunks, 2), 
         sub_val = map_chr(tmp_chunks, 3)) %>%
  select(-c(tmp_chunks, sub_value, sub_val))

head(res)
dim(res)

In [ ]:
sample_timepoint <- read.csv("../data/metadata/clin_data/sample_timepoint_formatted.csv")

In [ ]:
head(sample_timepoint)
sample_timepoint_days <- sample_timepoint %>% filter(phase == "phaseII")
sample_timepoint_days <- sample_timepoint_days %>% select(sampleID, sample_timepoint_days_since_OP)
sample_timepoint_days %>% filter(is.na(sample_timepoint_days_since_OP))
dim(sample_timepoint_days)
head(sample_timepoint_days)

In [ ]:
res$sampleID <- as.character(res$sampleID)
res <- left_join(res, sample_timepoint_days, by = c("sampleID"))
head(res)
dim(res)

In [ ]:
res %>% filter(is.na(sample_timepoint_days_since_OP))

In [ ]:
clinical_data <- read.csv("../data/metadata/clin_data/clinical_data_formatted.csv")
clinical_data_relapse <- clinical_data %>% select(patient_id, time_to_relapse_days, adjuvant_chemo_start_days, adjuvant_chemo_end_days)
head(clinical_data_relapse)

In [ ]:
res <-left_join(res, clinical_data_relapse, by="patient_id")
head(res)
dim(res)

In [ ]:
res_plot <- res %>% select(patient_id, cfDNA_folder, small_ratio, sample_timepoint_days_since_OP, time_to_relapse_days)
dim(res_plot)

In [ ]:
normalized_res_intervention <- data.frame(matrix(ncol = 7, nrow = 0))
colnames(normalized_res_intervention) <- c("patient_id", "cfDNA_folder", "small_ratio", "sample_timepoint_days_since_OP", "time_to_relapse_days","small_ratio_normalized_preop", "small_ratio_normalized_postop")

In [ ]:
head(normalized_res_intervention)

In [ ]:
patients = as.character(unique(res_plot$patient_id))

In [ ]:
for (i in patients){
    print(i)
    res_pt <- res_plot %>% filter(patient_id == i)
    
    first_sample <- min(res_pt$sample_timepoint_days_since_OP)
    without_first_sample <- res_pt %>% filter(sample_timepoint_days_since_OP != first_sample)
    post_op_sample <- min(without_first_sample$sample_timepoint_days_since_OP)
    
    for (x in res_pt$cfDNA_folder){
        sample_label_check <- res_pt %>% filter(cfDNA_folder == x)
        
        if (sample_label_check$sample_timepoint_days_since_OP == first_sample){
            #pre_val_big = sample_label_check$big_ratio
            #pre_val_jaccard = sample_label_check$jaccard
            pre_val_small_pre = sample_label_check$small_ratio}
        
        if (sample_label_check$sample_timepoint_days_since_OP == post_op_sample){
            #pre_val_big = sample_label_check$big_ratio
            #pre_val_jaccard = sample_label_check$jaccard
            pre_val_small_post = sample_label_check$small_ratio}
    }
    
    for (j in res_pt$cfDNA_folder){
        normalized_line <- res_pt %>% filter(cfDNA_folder == j)
        #print(normalized_line)
        normalized_line <- normalized_line %>% mutate(small_ratio_normalized_preop = small_ratio/pre_val_small_pre, small_ratio_normalized_postop = small_ratio/pre_val_small_post)
        #print(normalized_line)
        normalized_res_intervention[nrow(normalized_res_intervention)+1,] = c(i, j, normalized_line[3:7])
    
    }
}

In [ ]:
dim(normalized_res_intervention)

In [ ]:
length(unique(normalized_res_intervention$patient_id))

In [ ]:
#Correlations <- read.csv("../data/ci5_cs1e9_correlation.csv")
#head(Correlations)
#relapse_label <- Correlations %>% select(cfDNA_sample, Relapse)
#colnames(relapse_label) <- c("cfDNA_folder", "Relapse_label")

relapse <- read.csv("../data/metadata/clin_data//clinical_data_formatted.csv")
relapse <- relapse %>% filter(C2i_phase == "phaseII") %>% select(patient_id, is_relapse..0.no.1.yes., time_to_relapse_days)
relapse_label <- relapse %>% mutate(Relapse_label = ifelse(is_relapse..0.no.1.yes. == 0, "No_relapse", "Relapse")) %>% select(patient_id, Relapse_label)
head(relapse_label)

In [ ]:
dim(normalized_res_intervention)
normalized_res_intervention <- left_join(normalized_res_intervention, relapse_label, by = "patient_id")
head(normalized_res_intervention)
dim(normalized_res_intervention)

## Plots normalized by pre_op

In [ ]:
normalized_res_intervention_relapses <- normalized_res_intervention %>% filter(Relapse_label == "Relapse", sample_timepoint_days_since_OP >= time_to_relapse_days)
normalized_res_intervention_pre_or_no_relapse1 <- normalized_res_intervention %>% group_by(patient_id) %>% filter(Relapse_label == "Relapse", sample_timepoint_days_since_OP >= time_to_relapse_days) %>% filter(sample_timepoint_days_since_OP == min(sample_timepoint_days_since_OP))

normalized_res_intervention_pre_or_no_relapse2 <- normalized_res_intervention %>% filter(Relapse_label == "No_relapse" | 
                                                                                        sample_timepoint_days_since_OP < time_to_relapse_days)

normalized_res_intervention_pre_or_no_relapse <- bind_rows(normalized_res_intervention_pre_or_no_relapse1, normalized_res_intervention_pre_or_no_relapse2)

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

In [ ]:
#large_plot_big_r_normalized <- ggplot() + 
#    geom_line(data = normalized_res_intervention_pre_or_no_relapse, aes(x = sample_timepoint_days_since_OP, y = log10(big_ratio_normalized), color = Relapse_label, group=patient_id)) + 
#    geom_line(data = normalized_res_intervention_relapses, aes(x = sample_timepoint_days_since_OP, y = log10(big_ratio_normalized), color = Relapse_label, group=patient_id), linetype = "dashed") + 
#    geom_point(data = normalized_res_intervention, aes(x = sample_timepoint_days_since_OP, y = log10(big_ratio_normalized), color = Relapse_label))+ theme(text = element_text(size = 20))  
#ggsave(
#  paste("../plotting_results/", output_folder, "/big_plots/Relapse_noRelapse_Normalized_Big_", filetype, ".png", sep = ""),
#  large_plot_big_r_normalized, width = 12, height = 8, dpi = "print")

In [ ]:
#large_plot_big_r_normalized

In [ ]:
#large_plot_jaccard_normalized <- ggplot() + 
#    geom_line(data = normalized_res_intervention_pre_or_no_relapse, aes(x = sample_timepoint_days_since_OP, y = log10(jaccard_normalized), color = Relapse_label, group=patient_id)) + 
#    geom_line(data = normalized_res_intervention_relapses, aes(x = sample_timepoint_days_since_OP, y = log10(jaccard_normalized), color = Relapse_label, group=patient_id), linetype = "dashed") + 
#    geom_point(data = normalized_res_intervention, aes(x = sample_timepoint_days_since_OP, y = log10(jaccard_normalized), color = Relapse_label))+ theme(text = element_text(size = 20))  
#ggsave(
#  paste("../plotting_results/", output_folder, "/big_plots/Relapse_noRelapse_Normalized_Jaccard", filetype, ".png", sep = ""),
#  large_plot_jaccard_normalized, width = 12, height = 8, dpi = "print")

In [ ]:
#large_plot_jaccard_normalized

In [ ]:
large_plot_small_r_normalized <- ggplot() + 
    geom_line(data = normalized_res_intervention_pre_or_no_relapse, aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_preop), color = Relapse_label, group=patient_id)) + 
    geom_line(data = normalized_res_intervention_relapses, aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_preop), color = Relapse_label, group=patient_id), linetype = "dashed") + 
    geom_point(data = normalized_res_intervention, aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_preop), color = Relapse_label))+ theme(text = element_text(size = 20)) 

#ggsave(
#  paste("../plotting_results/", output_folder, "/big_plots/Relapse_noRelapse_Normalized_Small_", filetype, ".png", sep = ""),
#  large_plot_small_r_normalized, width = 12, height = 8, dpi = "print")

In [ ]:
large_plot_small_r_normalized

## Plots normalized by post-op

In [ ]:
#large_plot_big_r_normalized_by_post_op <- ggplot() + 
#    geom_line(data = normalized_res_post_op_pre_or_no_relapse %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(big_ratio_normalized), color = Relapse_label, group=patient_id), size = 1.1) + 
#    geom_line(data = normalized_res_post_op_relapses %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(big_ratio_normalized), color = Relapse_label, group=patient_id), linetype = "dashed", size = 1.1) + 
#    geom_point(data = normalized_res_post_op %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(big_ratio_normalized), color = Relapse_label), size = 3) + 
    
#    theme_minimal() + 
#    theme(text = element_text(size = 20))  + 
#    scale_color_manual(values=c('#00BFC4', '#F8766D'), labels = c("No relapse", "Relapse"))+
    

#    xlab("Sample time point (days since surgery)") + ylab("log10((UT in cfDNA / cfDNA) /(UT / T))")  
    #ggtitle("Estimated ctDNA levels in cfDNA, normalized by the first post-op sample (low quality sample removed)")


In [ ]:
#ggsave(
#  paste("../plotting_results/", output_folder, "/big_plots/Relapse_noRelapse_Normalized_by_post_op_Big_", filetype, ".png", sep = ""),
#  large_plot_big_r_normalized_by_post_op, width = 12, height = 8, dpi = "print")

In [ ]:
#large_plot_big_r_normalized_by_post_op

In [ ]:
#large_plot_jaccard_normalized_by_post_op <- ggplot() + 
#    geom_line(data = normalized_res_post_op_pre_or_no_relapse, aes(x = sample_timepoint_days_since_OP, y = log10(jaccard_normalized), color = Relapse_label, group=patient_id)) + 
#    geom_line(data = normalized_res_post_op_relapses, aes(x = sample_timepoint_days_since_OP, y = log10(jaccard_normalized), color = Relapse_label, group=patient_id), linetype = "dashed") + 
#    geom_point(data = normalized_res_post_op, aes(x = sample_timepoint_days_since_OP, y = log10(jaccard_normalized), color = Relapse_label))+ theme(text = element_text(size = 20))  
#ggsave(
#  paste("../plotting_results/", output_folder, "/big_plots/Relapse_noRelapse_Normalized_by_post_op_Jaccard_", filetype, ".png", sep = ""),
#  large_plot_jaccard_normalized_by_post_op, width = 12, height = 8, dpi = "print")

In [ ]:
#large_plot_jaccard_normalized_by_post_op

In [ ]:
ggplot() + 
    geom_line(data = normalized_res_intervention_pre_or_no_relapse %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = (small_ratio_normalized_postop), color = Relapse_label, group=patient_id)) + 
    geom_line(data = normalized_res_intervention_relapses %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = (small_ratio_normalized_postop), color = Relapse_label, group=patient_id), linetype = "dashed") + 
    geom_point(data = normalized_res_intervention %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = (small_ratio_normalized_postop), color = Relapse_label)) + 
    theme_minimal() + 
    theme(text = element_text(size = 20))  + 
    xlab("Sample time point (days since surgery)") + ylab("log10((UT kmers in cfDNA / UT kmers)") + 
    scale_color_manual(values=c('#00BFC4', '#F8766D'), labels = c("No relapse", "Relapse"))

In [ ]:
ggplot() + 
    geom_line(data = normalized_res_intervention_pre_or_no_relapse %>% filter(Relapse_label != "Relapse"), aes(x = sample_timepoint_days_since_OP, y = (small_ratio_normalized_postop), color = Relapse_label, group=patient_id)) + 
    geom_line(data = normalized_res_intervention_relapses %>% filter(Relapse_label != "Relapse"), aes(x = sample_timepoint_days_since_OP, y = (small_ratio_normalized_postop), color = Relapse_label, group=patient_id), linetype = "dashed") + 
    geom_point(data = normalized_res_intervention %>% filter(Relapse_label != "Relapse"), aes(x = sample_timepoint_days_since_OP, y = (small_ratio_normalized_postop), color = Relapse_label)) + 
    theme_minimal() + 
    theme(text = element_text(size = 20))  + 
    xlab("Sample time point (days since surgery)") + ylab("log10((UT kmers in cfDNA / UT kmers)") + 
    scale_color_manual(values=c('#00BFC4', '#F8766D'), labels = c("No relapse", "Relapse"))

In [ ]:
ggplot() + 
    geom_line(data = normalized_res_intervention_pre_or_no_relapse %>% filter(Relapse_label != "No_relapse"), aes(x = sample_timepoint_days_since_OP, y = (small_ratio_normalized_postop), color = Relapse_label, group=patient_id)) + 
    geom_line(data = normalized_res_intervention_relapses %>% filter(Relapse_label != "No_relapse"), aes(x = sample_timepoint_days_since_OP, y = (small_ratio_normalized_postop), color = Relapse_label, group=patient_id), linetype = "dashed") + 
    geom_point(data = normalized_res_intervention %>% filter(Relapse_label != "No_relapse"), aes(x = sample_timepoint_days_since_OP, y = (small_ratio_normalized_postop), color = Relapse_label)) + 
    theme_minimal() + 
    theme(text = element_text(size = 20))  + 
    xlab("Sample time point (days since surgery)") + ylab("log10((UT kmers in cfDNA / UT kmers)") + 
    scale_color_manual(values=c('#00BFC4', '#F8766D'), labels = c("No relapse", "Relapse"))

In [ ]:
ggplot() + 
    geom_line(data = normalized_res_intervention_pre_or_no_relapse %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_postop), color = Relapse_label, group=patient_id)) + 
    geom_line(data = normalized_res_intervention_relapses %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_postop), color = Relapse_label, group=patient_id), linetype = "dashed") + 
    geom_point(data = normalized_res_intervention %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_postop), color = Relapse_label)) + 
    theme_minimal() + 
    theme(text = element_text(size = 20))  + 
    xlab("Sample time point (days since surgery)") + ylab("log10((UT kmers in cfDNA / UT kmers)") + 
    scale_color_manual(values=c('#00BFC4', '#F8766D'), labels = c("No relapse", "Relapse"))

In [ ]:
large_plot_small_r_normalized_by_post_op <- ggplot() + 
    geom_line(data = normalized_res_intervention_pre_or_no_relapse %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_postop), color = Relapse_label, group=patient_id), size = 1.1) + 
    geom_line(data = normalized_res_intervention_relapses %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_postop), color = Relapse_label, group=patient_id), linetype = "dashed", size = 1.1) + 
    geom_point(data = normalized_res_intervention %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_postop), color = Relapse_label), size = 3) + 
    theme_minimal() + 
    theme(text = element_text(size = 20))  + 
    xlab("Sample time point (days since surgery)") + ylab("log10((UT kmers in cfDNA / UT kmers)") + 
    scale_color_manual(values=c('#00BFC4', '#F8766D'), labels = c("No relapse", "Relapse"))
    #scale_color_hue(labels = c("No relapse", "Relapse"))
    #ggtitle("Fraction of unique tumor k-mers found in cfDNA, normalized by the first post-op sample (low quality sample removed)")


In [ ]:
large_plot_small_r_normalized_by_post_op

In [ ]:
ggsave(
  paste("../plotting_results/", output_folder, "/big_plots/Relapse_noRelapse_Normalized_by_post_op_Small_", filetype, ".png", sep = ""),
  large_plot_small_r_normalized_by_post_op, width = 12, height = 8, dpi = "print")

In [ ]:
head(normalized_res_intervention_relapses)

## Plots not normalized

In [ ]:
#large_plot_big_r <- ggplot() + 
#    geom_line(data = normalized_res_intervention_pre_or_no_relapse, aes(x = sample_timepoint_days_since_OP, y = log10(big_ratio), color = Relapse_label, group=patient_id)) + 
#    geom_line(data = normalized_res_intervention_relapses, aes(x = sample_timepoint_days_since_OP, y = log10(big_ratio), color = Relapse_label, group=patient_id), linetype = "dashed") + 
#    geom_point(data = normalized_res_intervention, aes(x = sample_timepoint_days_since_OP, y = log10(big_ratio), color = Relapse_label))+ theme(text = element_text(size = 20)) +#+geom_text(data = normalized_res_intervention, aes(x = sample_timepoint_days_since_OP, y = big_ratio, label =cfDNA_folder)) 
#    theme_minimal()+
#    theme(text = element_text(size = 20)) + 
#    labs(color="Relapse status")+
#    xlab("Sample time point (days since surgery)") + ylab("log10(Estimate of ctDNA level in cfDNA)") +
#    scale_color_hue(labels = c("No relapse", "Relapse"))

In [ ]:
#large_plot_big_r

In [ ]:
#ggsave(
#  paste("../plotting_results/", output_folder, "/big_plots/Relapse_noRelapse_Big_", filetype, ".png", sep = ""),
#  large_plot_big_r, width = 12, height = 8, dpi = "print")

In [ ]:

#large_plot_jaccard <- ggplot() + 
#    geom_line(data = normalized_res_intervention_pre_or_no_relapse %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = jaccard, color = Relapse_label, group=patient_id)) + 
#    geom_line(data = normalized_res_intervention_relapses %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = jaccard, color = Relapse_label, group=patient_id), linetype = "dashed") + 
#    geom_point(data = normalized_res_intervention %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = jaccard, color = Relapse_label))+ theme(text = element_text(size = 20)) #+geom_text(data = normalized_res_intervention, aes(x = sample_timepoint_days_since_OP, y = big_ratio, label =cfDNA_folder)) 
#large_plot_jaccard

In [ ]:
head(normalized_res_intervention)

In [ ]:
options(repr.plot.width=20, repr.plot.height=8)
small_plot_big_r <- ggplot() + 
    geom_line(data = normalized_res_intervention_pre_or_no_relapse  %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_postop), color = Relapse_label, group=patient_id), size = 0.55) + 
    geom_line(data = normalized_res_intervention_relapses  %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_postop), color = Relapse_label, group=patient_id), linetype = "dashed", size = 0.55) + 
    geom_point(data = normalized_res_intervention  %>% filter(patient_id != low_qual_sample), aes(x = sample_timepoint_days_since_OP, y = log10(small_ratio_normalized_postop), color = Relapse_label), size = 2.2) + 
    theme_minimal() + 
    theme(text = element_text(size = 24),
          strip.text.x = element_blank()) + 
    xlab("Sample time point (days since surgery)") + ylab("log10((UT kmers in cfDNA / UT kmers)") +
    labs(color="Relapse status")+
    scale_color_hue(labels = c("No relapse", "Relapse"), name = "Relapse label") + facet_wrap(~Relapse_label, ncol = 1)
    #ggtitle("Fraction of unique tumor k-mers found in cfDNA (low quality sample removed)")



In [ ]:
ggsave(
  paste("../plotting_results/", output_folder, "/big_plots/Relapse_noRelapse_Small_", filetype, ".png", sep = ""),
  small_plot_big_r, width = 12, height = 8, dpi = "print")

In [ ]:
options(repr.plot.width=20, repr.plot.height=12)
small_plot_big_r